In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from eunjeon import Mecab

okt = Okt()
kkma = Kkma()
kom = Komoran()
mecab = Mecab()


import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import transformers

SyntaxError: invalid syntax (<ipython-input-112-e6109558c4ff>, line 1)

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [4]:
sample_submission=pd.read_csv("sample_submission.csv")

In [5]:
train1=train[['과제명','요약문_연구내용','label']]
test1=test[['과제명','요약문_연구내용']]

In [6]:
train2=train[['과제명','label']]
test2=test[['과제명']]

In [7]:
train_raw=train1['과제명'] + train1['요약문_연구내용']
test_raw=test1['과제명'] + test1['요약문_연구내용']

In [ ]:
# 과제명 + 요약문_연구내용
# 패딩을 40, 400으로 다르게 적용

In [69]:
temp=[]
temp2=[]
for i in range(0,len(train_raw)):
    temp.append(okt.nouns(str(train_raw[i])))
for i in range(0,len(test_raw)):
    temp2.append(okt.nouns(str(test_raw[i])))

In [70]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(temp)
tokenizer.fit_on_texts(temp2)

train_sequences=tokenizer.texts_to_sequences(temp)
test_sequences=tokenizer.texts_to_sequences(temp2)
word_vocab=tokenizer.word_index


In [71]:
train_inputs=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs=pad_sequences(test_sequences, maxlen=40, padding='post')

train_inputs2=pad_sequences(train_sequences, maxlen=400, padding='post')
test_inputs2=pad_sequences(test_sequences, maxlen=400, padding='post')

In [73]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"

In [76]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 32)            2127744   
                                                                 
 global_average_pooling1d_1   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 128)               4224      
                                                                 
 dense_3 (Dense)             (None, 46)                5934      
                                                                 
Total params: 2,137,902
Trainable params: 2,137,902
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
labels=np.array(train['label'])
len(set(labels))

46

In [78]:
# fit model
num_epochs = 30
history = model.fit(train_inputs, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4358/4358 [==============================] - 74s 16ms/step - loss: 0.8725 - accuracy: 0.8224 - val_loss: 0.7278 - val_accuracy: 0.8285
Epoch 2/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.6059 - accuracy: 0.8440 - val_loss: 0.6172 - val_accuracy: 0.8448
Epoch 3/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.4820 - accuracy: 0.8675 - val_loss: 0.5767 - val_accuracy: 0.8531
Epoch 4/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.4002 - accuracy: 0.8861 - val_loss: 0.5754 - val_accuracy: 0.8582
Epoch 5/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.3436 - accuracy: 0.8999 - val_loss: 0.5877 - val_accuracy: 0.8613
Epoch 6/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.2992 - accuracy: 0.9118 - val_loss: 0.6065 - val_accuracy: 0.8623
Epoch 7/30
4358/4358 [==============================] - 69s 16ms/step - loss: 0.2630 - accuracy: 0.9224 - val_loss: 0.6322 -

In [79]:
pred=model.predict(test_inputs)
pred=tf.argmax(pred, axis=1)

In [80]:
sample_submission['label']=pred
sample_submission.to_csv('패딩40 okt nouns.csv', index=False)

In [82]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 400
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 400, 32)           2127744   
                                                                 
 global_average_pooling1d_2   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 128)               4224      
                                                                 
 dense_5 (Dense)             (None, 46)                5934      
                                                                 
Total params: 2,137,902
Trainable params: 2,137,902
Non-trainable params: 0
_________________________________________________________________
None


In [84]:
# fit model
num_epochs = 30
history = model.fit(train_inputs2, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4358/4358 [==============================] - 74s 17ms/step - loss: 0.6698 - accuracy: 0.8337 - val_loss: 0.6353 - val_accuracy: 0.8373
Epoch 2/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.5347 - accuracy: 0.8564 - val_loss: 0.5613 - val_accuracy: 0.8525
Epoch 3/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.4441 - accuracy: 0.8751 - val_loss: 0.5246 - val_accuracy: 0.8623
Epoch 4/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.3781 - accuracy: 0.8908 - val_loss: 0.5010 - val_accuracy: 0.8705
Epoch 5/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.3282 - accuracy: 0.9038 - val_loss: 0.4989 - val_accuracy: 0.8744
Epoch 6/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.2888 - accuracy: 0.9144 - val_loss: 0.5010 - val_accuracy: 0.8777
Epoch 7/30
4358/4358 [==============================] - 71s 16ms/step - loss: 0.2559 - accuracy: 0.9241 - val_loss: 0.5110 -

In [85]:
pred2=model.predict(test_inputs2)
pred2=tf.argmax(pred2, axis=1)

In [86]:
sample_submission['label']=pred2
sample_submission.to_csv('패딩400 okt nouns.csv', index=False)

In [8]:
# 불용어 처리 다르게 적용
stop_w=pd.read_csv("stop_words.csv")

In [9]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_text=[token for token in word_text if not token in stop_words]
    return word_text

In [10]:
stop_words=list(stop_w)
clean_train_text=[]
clean_test_text=[]

In [11]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['과제명']):
    try:
        clean_train_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        clean_train_text.append([])

100%|█████████████████████████████████████████████████████████████████████████| 174304/174304 [23:59<00:00, 121.11it/s]


In [12]:
for text in tqdm.tqdm(test['과제명']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_text.append([])

100%|███████████████████████████████████████████████████████████████████████████| 43576/43576 [06:31<00:00, 111.41it/s]


In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text)

train_sequences=tokenizer.texts_to_sequences(clean_train_text)
test_sequences=tokenizer.texts_to_sequences(clean_test_text)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs3=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs3=pad_sequences(test_sequences, maxlen=40, padding='post')

In [14]:
labels=np.array(train['label'])
len(set(labels))

46

In [15]:
labels=np.array(train['label'])
len(set(labels))

#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 32)            1046464   
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dense_1 (Dense)             (None, 46)                5934      
                                                                 
Total params: 1,056,622
Trainable params: 1,056,622
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
num_epochs = 30
history = model.fit(train_inputs3, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4358/4358 [==============================] - 35s 7ms/step - loss: 0.9369 - accuracy: 0.8193 - val_loss: 0.8266 - val_accuracy: 0.8235
Epoch 2/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.6927 - accuracy: 0.8359 - val_loss: 0.6515 - val_accuracy: 0.8425
Epoch 3/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.5472 - accuracy: 0.8596 - val_loss: 0.5848 - val_accuracy: 0.8532
Epoch 4/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.4533 - accuracy: 0.8781 - val_loss: 0.5336 - val_accuracy: 0.8660
Epoch 5/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.3829 - accuracy: 0.8938 - val_loss: 0.5000 - val_accuracy: 0.8732
Epoch 6/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.3311 - accuracy: 0.9057 - val_loss: 0.4881 - val_accuracy: 0.8777
Epoch 7/30
4358/4358 [==============================] - 31s 7ms/step - loss: 0.2934 - accuracy: 0.9149 - val_loss: 0.4743 - val_ac

In [17]:
pred3=model.predict(test_inputs3)
pred3=tf.argmax(pred3, axis=1)

In [18]:
sample_submission['label']=pred3
sample_submission.to_csv('불용어 okt 패딩 40.csv', index=False)

In [ ]:
# Mecab 사용

In [30]:
temp_m1=[]
temp_m2=[]
for i in range(0,len(train2[['과제명']])):
    temp_m1.append(mecab.nouns(str(train2.iloc[i])))
for i in range(0,len(test2[['과제명']])):
    temp_m2.append(mecab.nouns(str(test2.iloc[i])))

In [34]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(temp_m1)
tokenizer.fit_on_texts(temp_m2)

train_sequences=tokenizer.texts_to_sequences(temp_m1)
test_sequences=tokenizer.texts_to_sequences(temp_m2)
word_vocab=tokenizer.word_index

train_inputs6=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs6=pad_sequences(test_sequences, maxlen=40, padding='post')

In [35]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 32)            852768    
                                                                 
 global_average_pooling1d_1   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_2 (Dense)             (None, 128)               4224      
                                                                 
 dense_3 (Dense)             (None, 46)                5934      
                                                                 
Total params: 862,926
Trainable params: 862,926
Non-trainable params: 0
_________________________________________________________________
None


In [36]:
num_epochs = 30
history = model.fit(train_inputs6, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.9234 - accuracy: 0.8206 - val_loss: 0.8035 - val_accuracy: 0.8248
Epoch 2/30
4358/4358 [==============================] - 26s 6ms/step - loss: 0.6638 - accuracy: 0.8412 - val_loss: 0.6301 - val_accuracy: 0.8483
Epoch 3/30
4358/4358 [==============================] - 26s 6ms/step - loss: 0.5247 - accuracy: 0.8639 - val_loss: 0.5517 - val_accuracy: 0.8607
Epoch 4/30
4358/4358 [==============================] - 27s 6ms/step - loss: 0.4435 - accuracy: 0.8786 - val_loss: 0.5114 - val_accuracy: 0.8682
Epoch 5/30
4358/4358 [==============================] - 27s 6ms/step - loss: 0.3842 - accuracy: 0.8920 - val_loss: 0.4864 - val_accuracy: 0.8761
Epoch 6/30
4358/4358 [==============================] - 27s 6ms/step - loss: 0.3403 - accuracy: 0.9028 - val_loss: 0.4734 - val_accuracy: 0.8795
Epoch 7/30
4358/4358 [==============================] - 27s 6ms/step - loss: 0.3068 - accuracy: 0.9105 - val_loss: 0.4806 - val_ac

In [37]:
pred6=model.predict(test_inputs6)
pred6=tf.argmax(pred6, axis=1)
sample_submission['label']=pred6
sample_submission.to_csv('mecab 명사.csv', index=False)

In [38]:
# Mecab 불용어

In [39]:
def preprocessing(text, mecab, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_text=[token for token in word_text if not token in stop_words]
    return word_text

stop_words=list(stop_w)
clean_train_text=[]
clean_test_text=[]

In [40]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['과제명']):
    try:
        clean_train_text.append(preprocessing(text, mecab, remove_stopwords=True, stop_words=stop_words))
    except:
        clean_train_text.append([])

100%|████████████████████████████████████████████████████████████████████████| 174304/174304 [1:03:51<00:00, 45.49it/s]


In [41]:
for text in tqdm.tqdm(test['과제명']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, mecab, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_text.append([])

100%|████████████████████████████████████████████████████████████████████████████| 43576/43576 [31:03<00:00, 23.39it/s]


In [42]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text)

train_sequences=tokenizer.texts_to_sequences(clean_train_text)
test_sequences=tokenizer.texts_to_sequences(clean_test_text)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs7=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs7=pad_sequences(test_sequences, maxlen=40, padding='post')

In [43]:
labels=np.array(train['label'])
len(set(labels))

#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 40, 32)            969152    
                                                                 
 global_average_pooling1d_2   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 128)               4224      
                                                                 
 dense_5 (Dense)             (None, 46)                5934      
                                                                 
Total params: 979,310
Trainable params: 979,310
Non-trainable params: 0
_________________________________________________________________
None


In [44]:
num_epochs = 30
history = model.fit(train_inputs7, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.9244 - accuracy: 0.8207 - val_loss: 0.8014 - val_accuracy: 0.8238
Epoch 2/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.6791 - accuracy: 0.8369 - val_loss: 0.6494 - val_accuracy: 0.8442
Epoch 3/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.5389 - accuracy: 0.8617 - val_loss: 0.5681 - val_accuracy: 0.8557
Epoch 4/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.4445 - accuracy: 0.8800 - val_loss: 0.5184 - val_accuracy: 0.8668
Epoch 5/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.3772 - accuracy: 0.8937 - val_loss: 0.4926 - val_accuracy: 0.8734
Epoch 6/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.3277 - accuracy: 0.9055 - val_loss: 0.4833 - val_accuracy: 0.8790
Epoch 7/30
4358/4358 [==============================] - 28s 6ms/step - loss: 0.2923 - accuracy: 0.9148 - val_loss: 0.4730 - val_ac

In [45]:
pred7=model.predict(test_inputs7)
pred7=tf.argmax(pred7, axis=1)
sample_submission['label']=pred7
sample_submission.to_csv('mecab 형태소 불용어.csv', index=False)

In [ ]:
######################################################################
########################## 아래는 아니에요 ###########################
######################################################################

In [ ]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 400
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

In [ ]:
train_inputs4=pad_sequences(train_sequences, maxlen=400, padding='post')
test_inputs4=pad_sequences(test_sequences, maxlen=400, padding='post')

In [ ]:
num_epochs = 30
history = model.fit(train_inputs4, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

In [ ]:
pred4=model.predict(test_inputs4)
pred4=tf.argmax(pred4, axis=1)

In [ ]:
sample_submission['label']=pred4
sample_submission.to_csv('패딩400 okt 형태소2.csv', index=False)

In [139]:
stop_w=pd.read_csv("stop_words.csv")

In [140]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ0-9]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_text=[token for token in word_text if not token in stop_words]
    return word_text

In [141]:
stop_words=list(stop_w)
okt=Okt()
clean_train_text2=[]
clean_test_text2=[]

In [143]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['과제명']):
    try:
        clean_train_text2.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        clean_train_text2.append([])

100%|████████████████████████████████████████████████████████████████████████| 174304/174304 [5:36:08<00:00,  8.64it/s]


In [144]:
for text in tqdm.tqdm(test['과제명']):
    if type(text) == str:
        clean_test_text2.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_text2.append([])

100%|██████████████████████████████████████████████████████████████████████████| 43576/43576 [1:28:13<00:00,  8.23it/s]


In [163]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text2)

train_sequences=tokenizer.texts_to_sequences(clean_train_text2)
test_sequences=tokenizer.texts_to_sequences(clean_test_text2)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs5=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs5=pad_sequences(test_sequences, maxlen=40, padding='post')

In [146]:
labels=np.array(train['label'])
len(set(labels))

46

In [162]:
test_inputs5.shape

(43576, 40)

In [164]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 40
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 40, 32)            1383872   
                                                                 
 global_average_pooling1d_7   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_14 (Dense)            (None, 128)               4224      
                                                                 
 dense_15 (Dense)            (None, 46)                5934      
                                                                 
Total params: 1,394,030
Trainable params: 1,394,030
Non-trainable params: 0
_________________________________________________________________
None


In [165]:
num_epochs = 30
history = model.fit(train_inputs5, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4585/4589 [============================>.] - ETA: 0s - loss: 1.0960 - accuracy: 0.8181

ValueError: Data cardinality is ambiguous:
  x sizes: 36708
  y sizes: 27475
Make sure all arrays contain the same number of samples.

In [ ]:
pred5=model.predict(test_inputs5)
pred5=tf.argmax(pred5, axis=1)

In [ ]:
sample_submission['label']=pred5
sample_submission.to_csv('패딩40 okt 불용어.csv', index=False)

In [150]:
#패딩 처리
train_inputs6=pad_sequences(train_sequences, maxlen=400, padding='post')
test_inputs6=pad_sequences(test_sequences, maxlen=400, padding='post')

In [151]:
labels=np.array(train['label'])
len(set(labels))

46

In [152]:
#파라미터 설정
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
vocab_size =data_configs['vocab_size']
embedding_dim = 32
max_length = 400
oov_tok = "<OOV>"
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 400, 32)           1383872   
                                                                 
 global_average_pooling1d_6   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 128)               4224      
                                                                 
 dense_13 (Dense)            (None, 46)                5934      
                                                                 
Total params: 1,394,030
Trainable params: 1,394,030
Non-trainable params: 0
_________________________________________________________________
None


In [153]:
num_epochs = 30
history = model.fit(train_inputs6, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

Epoch 1/30
4586/4589 [============================>.] - ETA: 0s - loss: 1.0925 - accuracy: 0.8180

ValueError: Data cardinality is ambiguous:
  x sizes: 36708
  y sizes: 27475
Make sure all arrays contain the same number of samples.

In [ ]:
pred6=model.predict(test_inputs6)
pred6=tf.argmax(pred6, axis=1)

In [ ]:
sample_submission['label']=pred6
sample_submission.to_csv('패딩400 okt 불용어.csv', index=False)

In [103]:
stop_data=list()
for i in range(0,len(stop_w)):
    stop_data.append(sw[i])

In [104]:
stop_data

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해'],
 ['을'],
 ['를'],
 ['에'],
 ['의'],
 ['가'],
 ['으로'],
 ['로'],
 ['에게'],
 ['뿐이다'],
 ['의거하여'],
 ['근거하여'],
 ['입각하여'],
 ['기준으로'],
 ['예하면'],
 ['예를 들면'],
 ['예를 들자면'],
 ['저'],
 ['소인'],
 ['소생'],
 ['저희'],
 ['지말고'],
 ['하지마'],
 ['하지마라'],
 ['다른'],
 ['물론'],
 ['또한'],
 ['그리고'],
 ['비길수 없다'],
 ['해서는 안된다'],
 ['뿐만 아니라'],
 ['만이 아니다'],
 ['만은 아니다'],
 ['막론하고'],
 ['관계없이'],
 ['그치지 않다'],
 ['그러나'],
 ['그런데'],
 ['하지만'],
 ['든간에'],
 ['논하지 않다'],
 ['따지지 않다'],
 ['설사'],
 ['비록'],
 ['더라도'],
 ['아니면'],
 ['만 못하다'],
 ['하는 편이 낫다'],
 ['불문하고'],
 ['향하여'],
 ['향해서'],
 ['향하다'],
 ['쪽으로'],
 ['틈타'],
 ['이용하여'],
 ['타다'],
 ['오르다'],
 ['제외하고'],
 ['이 외에'],
 ['이 밖에'],
 ['하여야'],
 ['비로소'],
 ['한다면 몰라도'],
 ['외에도'],
 ['이곳'],
 ['여기'],
 ['부터'],
 ['기점으로'],
 ['따라서'],
 ['할 생각이다'],
 ['하려고하다'],
 ['이리하여'],
 ['그리하여'],
 ['그렇게 함으로써'],
 ['하지만'],
 ['일때'],
 ['할때'],
 ['앞에서'],
 ['중에서'],
 ['보는데서'],
 ['으로써'],
 ['로써'],
 ['까지'],
 ['해야한다'],
 ['일것이다'],
 ['반드시'],
 ['할줄알다'],
 ['할수있다'],
 [

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(temp)
tokenizer.fit_on_texts(temp2)

train_sequences=tokenizer.texts_to_sequences(temp)
test_sequences=tokenizer.texts_to_sequences(temp2)
word_vocab=tokenizer.word_index

In [ ]:
word_vocab[1]

In [ ]:
train_inputs=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs=pad_sequences(test_sequences, maxlen=40, padding='post')

In [ ]:
data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1


In [ ]:
train_inputs = np.load('./data_in/train_input.npy')

In [ ]:
train_inputs

In [ ]:
def preprocessing(text, okt, remove_stopwords=False, stop_words=[]):
    text=re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]","", text)
    word_text=okt.morphs(text, stem=True)
    if remove_stopwords:
        word_text=[token for token in word_text if not token in stop_words]
    return word_text

In [ ]:
stop_words=['은','는','이','가', '하','아','것','들','의','있','되','수','보','주','등','한']
okt=Okt()
clean_train_text=[]
clean_test_text=[]

In [ ]:
#시간이 많이 걸립니다.
for text in tqdm.tqdm(train['과제명']):
    try:
        clean_train_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    except:
        clean_train_text.append([])

In [ ]:
for text in tqdm.tqdm(test['과제명']):
    if type(text) == str:
        clean_test_text.append(preprocessing(text, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_text.append([])

In [ ]:
#텐서플로의 전처리 모듈을 활용해 토크나이징 객체를 만든 후 인덱스 벡터로 전환
tokenizer=Tokenizer()
tokenizer.fit_on_texts(clean_train_text)

train_sequences=tokenizer.texts_to_sequences(clean_train_text)
test_sequences=tokenizer.texts_to_sequences(clean_test_text)
word_vocab=tokenizer.word_index

#패딩 처리
train_inputs=pad_sequences(train_sequences, maxlen=40, padding='post')
test_inputs=pad_sequences(test_sequences, maxlen=40, padding='post')

In [ ]:
print(train_inputs.shape)
print(test_inputs.shape)

In [ ]:
labels=np.array(train['label'])
len(set(labels))

In [ ]:
#추후 재사용 가능하도록 npy로 전환
DATA_IN_PATH='./data_in/'
TRAIN_INPUT_DATA = 'train_input.npy'
TEST_INPUT_DATA = 'test_input.npy'

import os
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)
    
np.save(open(DATA_IN_PATH+TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH+TEST_INPUT_DATA, 'wb'), test_inputs)

data_configs={}
data_configs['vocab']=word_vocab
data_configs['vocab_size'] = len(word_vocab)+1
json.dump(data_configs, open(DATA_IN_PATH+'data_configs.json', 'w'), ensure_ascii=False)

In [ ]:
#파라미터 설정
vocab_size =data_configs['vocab_size']
embedding_dim = 64
max_length = 40
oov_tok = "<OOV>"

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(46, activation='softmax')
])

# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

In [ ]:
# fit model
num_epochs = 30
history = model.fit(train_inputs, labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.2)

In [ ]:
#평가지표가 Macro F1이기에 확률값으로 결과를 내면 안됩니다.
pred=model.predict(test_inputs)
pred=tf.argmax(pred, axis=1)

sample_submission['label']=pred

sample_submission

In [ ]:
sample_submission.to_csv('baseline4.csv', index=False)

In [ ]:
# 랜덤포래스트

from sklearn.feature_extraction.text import CountVectorizer

#tokenizer 인자에는 list를 받아서 그대로 내보내는 함수를 넣어줍니다. 또한 소문자화를 하지 않도록 설정해야 에러가 나지 않습니다.
vectorizer = CountVectorizer(tokenizer = lambda x: x, lowercase=False)
train_features=vectorizer.fit_transform(clean_train_text)
test_features=vectorizer.transform(clean_test_text)
#test데이터에 fit_transform을 할 경우 data leakage에 해당합니다

In [ ]:
train_features

In [ ]:
test_features

In [ ]:
#훈련 데이터 셋과 검증 데이터 셋으로 분리
TEST_SIZE=0.2
RANDOM_SEED=42

train_x, eval_x, train_y, eval_y=train_test_split(train_features, train['label'], test_size=TEST_SIZE, random_state=RANDOM_SEED)

In [ ]:
#랜덤포레스트로 모델링
from sklearn.ensemble import RandomForestClassifier

forest=RandomForestClassifier(n_estimators=100)

forest.fit(train_x, train_y)

In [ ]:
#모델 검증
forest.score(eval_x, eval_y)

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15608201088283213939
 xla_global_id: -1]

In [3]:
pip freeze

- @ file:///C:/ci/importlib-metadata_1617877484576/work
-mportlib-metadata @ file:///C:/ci/importlib-metadata_1617877484576/work
absl-py==1.0.0
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client==1.7.2
anaconda-navigator==2.0.3
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
anyio @ file:///C:/ci/anyio_1620153418380/work/dist
appdirs==1.4.4
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1613037959010/work
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astroid @ file:///C:/ci/astroid_1613501047216/work
astropy @ file:///C:/ci/astropy_1617745647203/work
astunparse==1.6.3
async-generator @ file:///home/ktietz/src/ci/async_generator_1611927993394/work
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1615918855173/work
Babel @ file:///tmp/build/80754af9/babel_1607110387436/work
backcall @ file:///home/ktietz/src/ci/back


packaging @ file:///tmp/build/80754af9/packaging_1611952188834/work
pandas @ file:///C:/ci/pandas_1618365634936/work
pandocfilters @ file:///C:/ci/pandocfilters_1605102497129/work
paramiko @ file:///tmp/build/80754af9/paramiko_1598886428689/work
parso==0.7.0
partd @ file:///tmp/build/80754af9/partd_1618000087440/work
path @ file:///C:/ci/path_1614022440181/work
pathlib2 @ file:///C:/ci/pathlib2_1607025069150/work
pathspec==0.7.0
patsy==0.5.1
pep8==1.7.1
pexpect @ file:///tmp/build/80754af9/pexpect_1605563209008/work
pickleshare @ file:///tmp/build/80754af9/pickleshare_1606932040724/work
Pillow @ file:///C:/ci/pillow_1617386341487/work
pkginfo==1.7.0
pluggy @ file:///C:/ci/pluggy_1615976358795/work
ply==3.11
prometheus-client @ file:///tmp/build/80754af9/prometheus_client_1618088486455/work
prompt-toolkit @ file:///tmp/build/80754af9/prompt-toolkit_1616415428029/work
protobuf==3.19.3
psutil @ file:///C:/ci/psutil_1612298324802/work
ptyprocess @ file:///tmp/build/80754af9/ptyprocess_160